In [1]:
import cv2
import os
import numpy as np

def prepare_training_data(data_folder_path):
    image_files = os.listdir(data_folder_path)
    faces = []
    labels = []
    label = 1
    for image_name in image_files:
        if image_name.startswith("."):
            continue

        image_path = os.path.join(data_folder_path, image_name)
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        faces.append(gray)
        labels.append(label)
    return faces, labels

In [2]:
def detect_and_display(frame, face_recognizer):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        face = gray[y:y+h, x:x+w]
        label, confidence = face_recognizer.predict(face)

        confidence_threshold = 70
        label_text = "Unknown"
        if label == 1 and confidence < confidence_threshold:
            label_text = f"Andrey"

        cv2.putText(frame, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Video', frame)


In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
if face_cascade.empty():
    print("Ошибка загрузки модели Haar Cascade для обнаружения лиц")
    exit()

faces, labels = prepare_training_data("face_data")
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(faces, np.array(labels))

video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    print("Не удалось открыть веб-камеру")
    exit()

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    detect_and_display(frame, face_recognizer)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()